 **Loan approval**

importing packages

In [ ]:
import pandas as pd#for data manipulation
import matplotlib.pyplot as plt#for visualization
import seaborn as sns#for visualization
from sklearn.metrics import classification_report,confusion_matrix#for the calculation of efficiency
from sklearn.ensemble import RandomForestClassifier#for the model
from sklearn.model_selection import train_test_split,RandomizedSearchCV #for splitting the data into training and testing
import lime#for explaining the model
import lime.lime_tabular
import numpy as np#for mathematical operations
import shap #for explaining the model(not working)
from sklearn.ensemble import AdaBoostClassifier#for the model
from scipy.stats import randint#for hyperparameter tuning
from sklearn import preprocessing #for preprocessing
from sklearn.ensemble import GradientBoostingClassifier
from interpret import set_visualize_provider
from interpret.provider import InlineProvider
from interpret.glassbox import ExplainableBoostingClassifier
from interpret import show

In [ ]:
loan_df=pd.read_csv("loan_approval_dataset (1).csv")#reading the csv file
loan_df.head()#displaying the content of dataframe

In [ ]:
loan_df.columns#returns the columns present in dataframe

In [ ]:
loan_df.dtypes#returns the type of the instances present in the dataframe

-------------------------------------------------------------------------------------------------------------------------
preprocessing

In [ ]:
loan_df.isnull().sum()#checks whether null element is present or not ,if null is present .sum() calculates the total number of null elements

In [ ]:
loan_df.shape#returns the number of rows and columns in the dataframe

In [ ]:
loan_df.duplicated().sum()#checks whether duplicate is present or not

In [ ]:
loan_df.drop('loan_id',axis=1,inplace=True) #drops the loan_id column as it is not needed for the prediction process

In [ ]:
loan_df.rename(columns={' no_of_dependents':'No_of_dependents',' self_employed':'Self_Employed',' loan_term':'Loan_Term',' education':'Education',' income_annum':'Income_annum',' loan_amount':'Loan_Amount',' cibil_score':'Cibil_Score',' residential_assets_value':'Residential_Assets_Value',' commercial_assets_value':'Commercial_Assets_Value',' luxury_assets_value':'Luxury_Assets_Value',' bank_asset_value':'Bank_Asset_Value',' loan_status':'Loan_Status'},inplace=True)#renaming the columns

In [ ]:
label_encoder = preprocessing.LabelEncoder() #label encoding
obj = (loan_df.dtypes == 'object') #converting object to int
for col in list(obj[obj].index): 
  loan_df[col] = label_encoder.fit_transform(loan_df[col])#converting object to int
# To find the number of columns with  datatype==object 
obj = (loan_df.dtypes == 'object') 
print("Categorical variables:",len(list(obj[obj].index)))


In [ ]:
loan_df.dtypes#after conversion

--------------------------------------------------------------------------------------------------------------------
visualization

In [ ]:
plt.pie(loan_df.Loan_Status.value_counts(),labels=['Yes','No'],autopct='%1.1f%%',colors=['#BC7FCD','#FB9AD1'],explode=[0.1,0.1])#pie chart

from the above pie chart we can say that acceptance rate is greater

In [ ]:
plt.figure(figsize=(12,6))
sns.heatmap(loan_df.corr(),annot=True,mask=np.triu(loan_df.corr()),linewidths=0.5)#heatmap

since we know that correlation coefficients greater than 0.7 or less than -0.7 are considered strong.so if a value is greater than 0.7 or less than -0.7 then we can drop that

dropped columns:Bank_Asset_Value,Commercial_Assets_Value,Luxury_Assets_Value,Residential_Assets_Value,income_annum.

In [ ]:
loan_df=loan_df.drop(['Residential_Assets_Value','Commercial_Assets_Value','Luxury_Assets_Value','Bank_Asset_Value'],axis=1)#dropping the columns which have high corr() value

In [ ]:
sns.heatmap(loan_df.corr(),annot=True,mask=np.triu(loan_df.corr()),linewidths=0.7)#heatmap

---------------------------------------------------------------------------------------------------------------
*Splitting the dataset*

In [ ]:
X=loan_df.drop('Loan_Status',axis=1)#assign value to X and y
y=loan_df.Loan_Status

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.3,random_state=42)#splitting the data into 70% training and 30% testing

---------------------------------------------------------------------------------------------------------------------

**Randomforest**

In [ ]:
rfc = RandomForestClassifier(n_estimators = 100)#defining the model
rfc.fit(X_train,y_train)#fitting the model
y_pred_rfc=rfc.predict(X_test)#predicting the model
print(classification_report(y_test, y_pred_rfc))

In [ ]:
sns.heatmap(confusion_matrix(y_test, y_pred_rfc),fmt='d',annot=True)

In [ ]:
'''from xgboost import XGBClassifier
xgb = XGBClassifier(
    objective='multi:softmax',  # Objective for multi-class classification
    num_class=3,                 # Number of classes in the target variable
    eval_metric='mlogloss',      # Metric to evaluate performance
    use_label_encoder=False,     # Avoid label encoder warnings
    random_state=42
)

# Train the model
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_test)
print("classification report:\n", classification_report(y_test, y_pred_xgb))#classification_report(y_test, y_pred_xgb)
sns.heatmap(confusion_matrix(y_test, y_pred_xgb),fmt='d',annot=True)#confusion_matrix(y_test, y_pred_xgb)'''

--------------------------------------------------------------------------------------------------------------------

ADABOOST

In [ ]:
adaboost = AdaBoostClassifier(random_state=42) # Create an AdaBoost classifier  
param_dist = {
    'n_estimators': randint(50, 200),
    'learning_rate': [0.1, 0.2, 0.3, 0.4, 0.5],
    'algorithm': ['SAMME', 'SAMME.R']
}# Hyperparameter grid for RandomizedSearchCV
random_search = RandomizedSearchCV(adaboost, param_distributions=param_dist, n_iter=10, cv=5, n_jobs=-1, random_state=42)
random_search.fit(X_train, y_train) # Fit the RandomizedSearchCV object to the data
print("Best parameters:", random_search.best_params_)
print("Best score:", random_search.best_score_)
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)
print(classification_report(y_pred,y_test))

--------------------------------------------------------------------------------------------------------------------

**Gradient Boosting**

In [36]:
gbc = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1)# Create a Gradient Boosting classifier   
gbc.fit(X_train, y_train)# Train the model
y_pred_gbc = gbc.predict(X_test)# Make predictions
print(classification_report(y_test, y_pred_gbc))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       810
           1       0.98      0.97      0.98       471

    accuracy                           0.98      1281
   macro avg       0.98      0.98      0.98      1281
weighted avg       0.98      0.98      0.98      1281



----------------------------------------------------------------------------------------------------------------------

**XAI**(Explainable AI)

lime

In [ ]:
X_train_array=X_train.to_numpy()#converting the dataframe to array so we can add it to explainer of lime
explainer = lime.lime_tabular.LimeTabularExplainer(
    training_data=X_train_array,
    feature_names=X_train.columns,
    #class_names=loan_dropped_df.Loan_Status,
    mode='classification'
)#creating the explainer

In [ ]:
instance =X_test.iloc[0];#selecting an instance

In [ ]:
explanation = explainer.explain_instance(
    data_row=instance,
    predict_fn=rfc.predict_proba
)#creating the explanation


In [ ]:
explanation.show_in_notebook()#showing the explanation

Coming to the visualizations, we can see the colors blue and orange, depicting negative and positive associations, respectively.
First plot from left explains the prediction probability of loan status.Next plot explains what are the required criteria for loan approval. last column shows the features in the dataset(which was given to the model to predict).

----------------------------------------------------------------------------------------------------------------------------------
EBM

Explainable Boosting Machine (EBM) is a tree-based, cyclic gradient boosting Generalized Additive Model with automatic interaction detection. EBMs are often as accurate as state-of-the-art blackbox models while remaining completely interpretable.

In [40]:
set_visualize_provider(InlineProvider())

In [ ]:
ebm = ExplainableBoostingClassifier()
ebm.fit(X_train, y_train)

In [42]:
show(ebm.explain_global())

In [43]:
show(ebm.explain_local(X_test[:5], y_test[:5]), 0)

----------------------------------------------------------------------------------------------------------------

ELI5

In [ ]:
##to install this 
#pip install eli5

In [44]:
import eli5#giving importError
#eli5.show_weights(gbc, feature_names=X_test.columns.tolist())( if it doesnt give error run this)


ImportError: cannot import name 'if_delegate_has_method' from 'sklearn.utils.metaestimators' (c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\metaestimators.py)